# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 25 2022 8:49AM,238037,10,0085712554,ISDIN Corporation,Released
1,May 25 2022 8:49AM,238037,10,0085712596,ISDIN Corporation,Released
2,May 25 2022 8:49AM,238037,10,0085712597,ISDIN Corporation,Released
3,May 25 2022 8:49AM,238037,10,0085712629,ISDIN Corporation,Released
4,May 25 2022 8:49AM,238037,10,0085712635,ISDIN Corporation,Released
5,May 25 2022 8:49AM,238037,10,0085712634,ISDIN Corporation,Released
6,May 25 2022 8:49AM,238037,10,0085712638,ISDIN Corporation,Released
7,May 25 2022 8:49AM,238037,10,0085712641,ISDIN Corporation,Released
8,May 25 2022 8:49AM,238037,10,0085712636,ISDIN Corporation,Released
9,May 25 2022 8:49AM,238037,10,0085712640,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,238031,Released,1
28,238033,Released,2
29,238034,Released,1
30,238036,Released,1
31,238037,Released,20


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238031,NaN,NaN,1.0
238033,NaN,NaN,2.0
238034,NaN,NaN,1.0
238036,NaN,NaN,1.0
238037,NaN,NaN,20.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237856,0.0,1.0,0.0
237864,0.0,2.0,9.0
237961,0.0,0.0,1.0
237965,0.0,0.0,1.0
237975,3.0,13.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237856,0,1,0
237864,0,2,9
237961,0,0,1
237965,0,0,1
237975,3,13,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237856,0,1,0
1,237864,0,2,9
2,237961,0,0,1
3,237965,0,0,1
4,237975,3,13,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237856,,1,
1,237864,,2,9
2,237961,,,1
3,237965,,,1
4,237975,3,13,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 25 2022 8:49AM,238037,10,ISDIN Corporation
20,May 25 2022 8:47AM,238036,10,"Amcyte Pharma, Inc."
21,May 25 2022 8:46AM,238033,18,Eye Pharma Inc
23,May 25 2022 8:43AM,238034,10,Emerginnova
24,May 25 2022 8:11AM,238031,19,"Dr. Raju's Pharma Solutions, LLC"
25,May 25 2022 8:09AM,238028,18,Hush Hush
44,May 25 2022 8:05AM,238030,22,"NBTY Global, Inc."
45,May 25 2022 8:04AM,238029,19,ACG North America LLC
46,May 25 2022 8:03AM,238027,18,Hush Hush
47,May 25 2022 12:00AM,238012,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 25 2022 8:49AM,238037,10,ISDIN Corporation,,,20
1,May 25 2022 8:47AM,238036,10,"Amcyte Pharma, Inc.",,,1
2,May 25 2022 8:46AM,238033,18,Eye Pharma Inc,,,2
3,May 25 2022 8:43AM,238034,10,Emerginnova,,,1
4,May 25 2022 8:11AM,238031,19,"Dr. Raju's Pharma Solutions, LLC",,,1
5,May 25 2022 8:09AM,238028,18,Hush Hush,,,19
6,May 25 2022 8:05AM,238030,22,"NBTY Global, Inc.",,,1
7,May 25 2022 8:04AM,238029,19,ACG North America LLC,,,1
8,May 25 2022 8:03AM,238027,18,Hush Hush,,,1
9,May 25 2022 12:00AM,238012,10,ISDIN Corporation,,1,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 8:49AM,238037,10,ISDIN Corporation,20,,
1,May 25 2022 8:47AM,238036,10,"Amcyte Pharma, Inc.",1,,
2,May 25 2022 8:46AM,238033,18,Eye Pharma Inc,2,,
3,May 25 2022 8:43AM,238034,10,Emerginnova,1,,
4,May 25 2022 8:11AM,238031,19,"Dr. Raju's Pharma Solutions, LLC",1,,
5,May 25 2022 8:09AM,238028,18,Hush Hush,19,,
6,May 25 2022 8:05AM,238030,22,"NBTY Global, Inc.",1,,
7,May 25 2022 8:04AM,238029,19,ACG North America LLC,1,,
8,May 25 2022 8:03AM,238027,18,Hush Hush,1,,
9,May 25 2022 12:00AM,238012,10,ISDIN Corporation,10,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 8:49AM,238037,10,ISDIN Corporation,20,,
1,May 25 2022 8:47AM,238036,10,"Amcyte Pharma, Inc.",1,,
2,May 25 2022 8:46AM,238033,18,Eye Pharma Inc,2,,
3,May 25 2022 8:43AM,238034,10,Emerginnova,1,,
4,May 25 2022 8:11AM,238031,19,"Dr. Raju's Pharma Solutions, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 8:49AM,238037,10,ISDIN Corporation,20.0,NaN,NaN
1,May 25 2022 8:47AM,238036,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN
2,May 25 2022 8:46AM,238033,18,Eye Pharma Inc,2.0,NaN,NaN
3,May 25 2022 8:43AM,238034,10,Emerginnova,1.0,NaN,NaN
4,May 25 2022 8:11AM,238031,19,"Dr. Raju's Pharma Solutions, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 8:49AM,238037,10,ISDIN Corporation,20.0,0.0,0.0
1,May 25 2022 8:47AM,238036,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0
2,May 25 2022 8:46AM,238033,18,Eye Pharma Inc,2.0,0.0,0.0
3,May 25 2022 8:43AM,238034,10,Emerginnova,1.0,0.0,0.0
4,May 25 2022 8:11AM,238031,19,"Dr. Raju's Pharma Solutions, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1428144,36.0,1.0,0.0
16,475986,2.0,0.0,0.0
18,1665901,24.0,2.0,0.0
19,1428039,7.0,19.0,3.0
20,713831,41.0,16.0,10.0
21,237980,1.0,0.0,0.0
22,238030,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1428144,36.0,1.0,0.0
1,16,475986,2.0,0.0,0.0
2,18,1665901,24.0,2.0,0.0
3,19,1428039,7.0,19.0,3.0
4,20,713831,41.0,16.0,10.0
5,21,237980,1.0,0.0,0.0
6,22,238030,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,36.0,1.0,0.0
1,16,2.0,0.0,0.0
2,18,24.0,2.0,0.0
3,19,7.0,19.0,3.0
4,20,41.0,16.0,10.0
5,21,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,36.0
1,16,Released,2.0
2,18,Released,24.0
3,19,Released,7.0
4,20,Released,41.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,3.0,10.0,0.0,0.0
Executing,1.0,0.0,2.0,19.0,16.0,0.0,0.0
Released,36.0,2.0,24.0,7.0,41.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,3.0,10.0,0.0,0.0
1,Executing,1.0,0.0,2.0,19.0,16.0,0.0,0.0
2,Released,36.0,2.0,24.0,7.0,41.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,3.0,10.0,0.0,0.0
1,Executing,1.0,0.0,2.0,19.0,16.0,0.0,0.0
2,Released,36.0,2.0,24.0,7.0,41.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()